## Failure!!! Dictionary를 이용하면 시간이 절약.. 매우 많이 적약됨..
### ref file match and change 
plink bim 파일에서 A1, A2를 ref 파일내에 있는 allele과 비교하여 매칭되는 allele이 ref allele입니다.
매칭 후 chr:pos_ref/alt 로 bim 파일내에 있는 affy id 를 변환하여
1KG P3와 공통인 마커만 추려 합친 후 PCA 분석을 진행하면 됩니다.
<분석과정>
1. plink bim 파일 A1, A2에서 ref allele 확인
2. bim 파일 내 affy id를 new id(chr:pos_ref/alt)로 변환
3. 1KG P3와 공통인 마커 추린 후 두 데이터를 합치기
4. PCA 분석하기
5. 인종별 색을 다르게 하여 plot 그리기 (1000GP_Phase3.sample에서 인종별 ID 확인)

In [1]:
import os
import pandas as pd

In [2]:
wdir = "c:/Users/user/Desktop/KCDC/Gastric/Ref/"

In [3]:
def fileRead(fileIn):
    f = open(fileIn,'r')
    inData = [r.replac("r","").replace("\n","").replace("\t","") for r in f]
    return inData

In [33]:

bim = pd.read_csv(wdir+"Case_Control_merge_rmfreq.bim",delim_whitespace = True,header = None)
ref = pd.read_csv(wdir+"Axiom_KOR.annot.extract.addINDEL.Final.REF.txt",delim_whitespace = True,header = None)

In [5]:
print(bim.head())
print(ref.head())

   0            1  2       3  4  5
0  1  AX-83277131  0  762485  A  C
1  1  AX-64175821  0  792643  G  A
2  1  AX-32415015  0  834956  A  G
3  1  AX-32416607  0  835499  G  A
4  1  AX-32429231  0  839461  C  T
                 0  1
0  AFFX-KIT-000001  T
1  AFFX-KIT-000002  G
2  AFFX-KIT-000003  T
3  AFFX-KIT-000004  G
4  AFFX-KIT-000005  T


In [6]:
print(ref.loc[ref[0] == bim[1][1]])
print(ref.loc[ref[0] == bim[1][2]])


                  0  1
859061  AX-64175821  A
                  0  1
670128  AX-32415015  G


In [25]:
len(bim),len(ref)

(486508, 903111)

In [48]:
ref.index = ref[0]
name_list = list(bim[1])
new_ref = ref.loc[name_list]
new_ref = new_ref.reset_index(drop = True)

In [ ]:
new_id = []
for i in range(0,len(bim)):
    if i%10000 == 0:
        print(i)
    index = new_ref.loc[ref[0] == bim[1][i]]
    if (index[1] == bim[4][i]).bool():
        new_id.append(str(bim[0][i]) + ":" + str(bim[3][i]) +"_"+str(bim[4][i])+"/"+str(bim[5][i]))
    else:
        new_id.append(str(bim[0][i]) + ":" + str(bim[3][i]) +"_"+str(bim[5][i])+"/"+str(bim[4][i]))
        tmp = bim.loc[i,5]
        bim.loc[i,5] = bim.loc[i,4]
        bim.loc[i,4] = tmp
        
bim.loc[:,1] = new_id
bim.to_csv(wdir+"match_ref_merge.bim",header=False,index = False,sep='\t')


10000


In [50]:
len(new_ref)

486508

In [9]:
bim = pd.read_csv(wdir+"Case_Control_merge_rmfreq.bim",delim_whitespace = True,header = None)
#bim.columns = ["CHR","MARKER_ID","FAM_ID","POS","REF","ALT"]

new_id = []
toy = bim[0:10]
print(toy)
for i in range(0,len(toy)):
    #try:
        #print(ref.loc[ref[0] == bim[1][i]])
        index = ref.loc[ref[0] == toy[1][i]]
       # print(index[1] == toy[4][i])
        #index[1] == bim[4][i]
        
        if (index[1] == toy[4][i]).bool():
            new_id.append(str(toy[0][i]) + ":" + str(toy[3][i]) +"_"+str(toy[4][i])+"/"+str(toy[5][i]))
        else:
            new_id.append(str(toy[0][i]) + ":" + str(toy[3][i]) +"_"+str(toy[5][i])+"/"+str(toy[4][i])) 
            
            tmp = toy.loc[i,4]
            toy.loc[i,4] = toy.loc[i,5]
            toy.loc[i,5] = tmp
    #    continue
print(new_id)
toy.loc[:,1] = new_id
toy

   0             1  2       3  4  5
0  1   AX-83277131  0  762485  A  C
1  1   AX-64175821  0  792643  G  A
2  1   AX-32415015  0  834956  A  G
3  1   AX-32416607  0  835499  G  A
4  1   AX-32429231  0  839461  C  T
5  1   AX-32455597  0  848456  G  A
6  1   AX-32460873  0  850371  G  T
7  1   AX-32461903  0  850780  C  T
8  1  AX-113228954  0  852820  C  T
9  1   AX-32469373  0  852964  T  G
['1:762485_C/A', '1:792643_A/G', '1:834956_G/A', '1:835499_A/G', '1:839461_T/C', '1:848456_A/G', '1:850371_G/T', '1:850780_C/T', '1:852820_T/C', '1:852964_T/G']


,0,1,2,3,4,5
0,1,1:762485_C/A,0,762485,C,A
1,1,1:792643_A/G,0,792643,A,G
2,1,1:834956_G/A,0,834956,G,A
3,1,1:835499_A/G,0,835499,A,G
4,1,1:839461_T/C,0,839461,T,C
5,1,1:848456_A/G,0,848456,A,G
6,1,1:850371_G/T,0,850371,G,T
7,1,1:850780_C/T,0,850780,C,T
8,1,1:852820_T/C,0,852820,T,C
9,1,1:852964_T/G,0,852964,T,G


In [44]:
bim = pd.read_csv(wdir+"Case_Control_merge_rmfreq.bim",delim_whitespace = True,header = None)
ref = pd.read_csv(wdir+"Axiom_KOR.annot.extract.addINDEL.Final.REF.txt",delim_whitespace = True,header = None)

toy = bim[1:20]
name_list = list(toy[1])
ref.index = ref[0]
#ref.set_index(0)
ref.head()
toy_ref = ref.loc[name_list]
print(toy_ref)
toy_ref.reset_index(drop =True)


                         0  1
0                            
AX-64175821    AX-64175821  A
AX-32415015    AX-32415015  G
AX-32416607    AX-32416607  A
AX-32429231    AX-32429231  T
AX-32455597    AX-32455597  A
AX-32460873    AX-32460873  G
AX-32461903    AX-32461903  C
AX-113228954  AX-113228954  T
AX-32469373    AX-32469373  T
AX-32501941    AX-32501941  A
AX-32515369    AX-32515369  C
AX-32519659    AX-32519659  G
AX-32532271    AX-32532271  C
AX-96723061    AX-96723061  C
AX-113404132  AX-113404132  C
AX-40371761    AX-40371761  A
AX-32590017    AX-32590017  G
AX-40386941    AX-40386941  T
AX-32598061    AX-32598061  C


,0,1
0,AX-64175821,A
1,AX-32415015,G
2,AX-32416607,A
3,AX-32429231,T
4,AX-32455597,A
5,AX-32460873,G
6,AX-32461903,C
7,AX-113228954,T
8,AX-32469373,T
9,AX-32501941,A


In [23]:
toy_ref

,0,1
0,,
AX-64175821,AX-64175821,A
AX-32415015,AX-32415015,G
AX-32416607,AX-32416607,A
AX-32429231,AX-32429231,T
AX-32455597,AX-32455597,A
AX-32460873,AX-32460873,G
AX-32461903,AX-32461903,C
AX-113228954,AX-113228954,T
AX-32469373,AX-32469373,T
